# Machine Perception Demo

The Machine Perception tool takes in arbitrary video streams and converts them to metric depth images, while segmenting obstacles and providing physical distances to them.

**Input:**
- A video sequence
- Camera intrinsics
- Height of the camera above a ground plane

**Output:**
- Metric depth maps
- Panoptic segmentation of the scene
- Bounding boxes with physical distances to anomolies and obstacles in the scene.

**Future Development**
- Cloud tool for remote monitoring of camera streams and depth + predictions
- Automatically fine-tuned models

***

## Code
**Import libraries for Detectron2**

In [1]:
###!python -m detectron2.utils.collect_env # to check if Detectron2 is working fine
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import open3d as o3d
from MiDaS.utils import read_pfm
from MiDaS.run import run

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

**Import libraries and files for MonoDepth2 algorithm**

In [2]:
# Libraries for monodepth2
from __future__ import absolute_import, division, print_function
%matplotlib inline

import os
import PIL.Image as pil
import glob

import torch
from torchvision import transforms

#import monodepth2.networks as networks
#from monodepth2.utils import download_model_if_doesnt_exist
#from monodepth2.layers import disp_to_depth
import matplotlib as mpl
import matplotlib.cm as cm

**Define key variables**

In [3]:
frames_folder = 'frames'
result_folder = 'results'
depths_folder = 'results/depth'
detections_folder = 'results/frames'
frame_count = None # Number of frames to consider in the video (use less for faster calculations) # None will take all frames
img_ext = 'png'

**Convert Video to PNG Frames**

In [4]:
!rm -rf $frames_folder/*
!mkdir $frames_folder/
!rm -rf $detections_folder/*
!mkdir $detections_folder/

#specify path to video
video = "outputApple.mp4"

#capture video
cap = cv2.VideoCapture(video)
cnt=0
FPS=cap.get(cv2.CAP_PROP_FPS)
freq = 0.1 #Num of desired seconds
# Check if video file is opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

ret,first_frame = cap.read()

#Read until video is completed
with tqdm(total=frame_count) as pbar:
    while(cap.isOpened()):

      # Capture frame-by-frame
      ret, frame = cap.read()
      pbar.update(1)
      if ret == True:

        #save each frame to folder        
        cv2.imwrite(frames_folder+'/{:04d}'.format(cnt)+'.png', frame)
        cnt=int(cnt+FPS*freq)
        cap.set(1,cnt)
        if(cnt==frame_count) and frame_count != None:
          break
      # Break the loop
      else: 
        break


mkdir: cannot create directory ‘frames/’: File exists
mkdir: cannot create directory ‘results/frames/’: File exists


188it [00:34,  5.41it/s]


**Loading MonoDept2 pretrained model**

In [5]:

if torch.cuda.is_available(): # and not args.no_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

"""
model_name = "mono+stereo_640x192"
download_model_if_doesnt_exist(model_name)
encoder_path = os.path.join("monodepth2/models", model_name, "encoder.pth")
depth_decoder_path = os.path.join("monodepth2/models", model_name, "depth.pth")


# LOADING PRETRAINED MODEL
encoder = networks.ResnetEncoder(18, False)
loaded_dict_enc = torch.load(encoder_path, map_location=device)
filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}
encoder.load_state_dict(filtered_dict_enc)

# extract the height and width of image that this model was trained with
feed_height = loaded_dict_enc['height']
feed_width = loaded_dict_enc['width']


encoder.to(device)
encoder.eval();

# LOADING PRETRAINED MODEL
depth_decoder = networks.DepthDecoder(num_ch_enc=encoder.num_ch_enc, scales=range(4))
loaded_dict = torch.load(depth_decoder_path, map_location=device)
depth_decoder.load_state_dict(loaded_dict)

depth_decoder.to(device)
depth_decoder.eval();
"""

'\nmodel_name = "mono+stereo_640x192"\ndownload_model_if_doesnt_exist(model_name)\nencoder_path = os.path.join("monodepth2/models", model_name, "encoder.pth")\ndepth_decoder_path = os.path.join("monodepth2/models", model_name, "depth.pth")\n\n\n# LOADING PRETRAINED MODEL\nencoder = networks.ResnetEncoder(18, False)\nloaded_dict_enc = torch.load(encoder_path, map_location=device)\nfiltered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}\nencoder.load_state_dict(filtered_dict_enc)\n\n# extract the height and width of image that this model was trained with\nfeed_height = loaded_dict_enc[\'height\']\nfeed_width = loaded_dict_enc[\'width\']\n\n\nencoder.to(device)\nencoder.eval();\n\n# LOADING PRETRAINED MODEL\ndepth_decoder = networks.DepthDecoder(num_ch_enc=encoder.num_ch_enc, scales=range(4))\nloaded_dict = torch.load(depth_decoder_path, map_location=device)\ndepth_decoder.load_state_dict(loaded_dict)\n\ndepth_decoder.to(device)\ndepth_decoder.eval();\n

**Performing depth estimation**

In [6]:
def findDepth(image_path,output_directory,ext):
    # FINDING INPUT IMAGES
    if os.path.isfile(image_path):
        # Only testing on a single image
        paths = [image_path]
        #output_directory = os.path.dirname(args.image_path)
    elif os.path.isdir(image_path):
        # Searching folder for images
        paths = glob.glob(os.path.join(image_path, '*.{}'.format(ext)))
        #output_directory = args.image_path
    else:
        raise Exception("Can not find args.image_path: {}".format(image_path))
    #print("-> Predicting on {:d} test images".format(len(paths)))


    # PREDICTING ON EACH IMAGE IN TURN
    with torch.no_grad():
        with tqdm(total=len(paths)) as pbar:
            for idx, image_path in (enumerate(paths)):

                if image_path.endswith("_disp.jpg"):
                    # don't try to predict disparity for a disparity image!
                    continue

                # Load image and preprocess
                input_image = pil.open(image_path).convert('RGB')
                original_width, original_height = input_image.size
                input_image = input_image.resize((feed_width, feed_height), pil.LANCZOS)
                input_image = transforms.ToTensor()(input_image).unsqueeze(0)

                # PREDICTION
                input_image = input_image.to(device)
                features = encoder(input_image)
                outputs = depth_decoder(features)

                disp = outputs[("disp", 0)]
                disp_resized = torch.nn.functional.interpolate(
                    disp, (original_height, original_width), mode="bilinear", align_corners=False)

                print("Output of network resized:")
                print(np.shape(disp_resized))
                # Saving numpy file
                output_name = os.path.splitext(os.path.basename(image_path))[0]
                name_dest_npy = os.path.join(output_directory, "{}_disp.npy".format(output_name))
                _, scaled_disp = disp_to_depth(disp, 3.0, 5000)
                np.save(name_dest_npy, scaled_disp.cpu().numpy())
                
                print("Absolute depth size:")
                print(np.shape(scaled_disp))
                #print(np.shape(disp))
                #break
                
                # Saving colormapped depth image
                disp_resized_np = disp_resized.squeeze().cpu().numpy()
                
                #print(disp_resized_np)
                
                vmax = np.percentile(disp_resized_np, 95)
                normalizer = mpl.colors.Normalize(vmin=disp_resized_np.min(), vmax=vmax)
                
                mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
                colormapped_im = (mapper.to_rgba(disp_resized_np)[:, :, :3] * 255).astype(np.uint8)
                
                #print(np.shape(colormapped_im))
                #break
                
                im = pil.fromarray(colormapped_im)
                
                name_dest_im = os.path.join(output_directory, "{}_disp.jpeg".format(output_name))
                im.save(name_dest_im)
                pbar.update(1)

In [7]:
!rm -rf $depths_folder/*
!mkdir $depths_folder
run(frames_folder,depths_folder,'./MiDaS/model-f6b98070.pt')
intrinsic = o3d.io.read_pinhole_camera_intrinsic("intrinsics.json")

c_imgs = glob.glob(frames_folder+"/*.png")
c_imgs.sort()

d_imgs = glob.glob(depths_folder+"/*.pfm")
d_imgs.sort()

h = 2.5

for idx in range(len(c_imgs)):
    color = o3d.io.read_image(c_imgs[idx])
    idepth = read_pfm(d_imgs[idx])[0]
    idepth = idepth - np.amin(idepth)
    idepth[idepth < 0.1] = 0.1
    idepth /= np.amax(idepth)
    idepth *= 10

    focal = intrinsic.intrinsic_matrix[0, 0]
    print(intrinsic.intrinsic_matrix)
    depth = focal / (idepth)
    #depth[depth >= 100 * focal] = np.inf

    imgDepth = o3d.geometry.Image(depth)

    rgbdi = o3d.geometry.RGBDImage.create_from_color_and_depth(color, imgDepth)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdi, intrinsic)
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    m = -h*b/d

    depth = (depth*m)/1000.0
    print (np.amin(depth))
    print (np.amax(depth))
    print (np.median(depth))
    np.save(depths_folder+f"/{idx}.npy", depth)
    #o3d.io.write_point_cloud(f"{idx}.npy", pcd)


#findDepth(frames_folder,depths_folder,img_ext)

mkdir: cannot create directory ‘results/depth’: File exists
initialize
device: cuda
Loading weights:  ./MiDaS/model-f6b98070.pt


Using cache found in /home/awsgui/.cache/torch/hub/facebookresearch_WSL-Images_master


start processing
  processing frames/0117.png (1/187)
  processing frames/0134.png (2/187)
  processing frames/0120.png (3/187)
  processing frames/0077.png (4/187)
  processing frames/0177.png (5/187)
  processing frames/0058.png (6/187)
  processing frames/0018.png (7/187)
  processing frames/0169.png (8/187)
  processing frames/0031.png (9/187)
  processing frames/0025.png (10/187)
  processing frames/0010.png (11/187)
  processing frames/0136.png (12/187)
  processing frames/0040.png (13/187)
  processing frames/0133.png (14/187)
  processing frames/0099.png (15/187)
  processing frames/0125.png (16/187)
  processing frames/0161.png (17/187)
  processing frames/0122.png (18/187)
  processing frames/0030.png (19/187)
  processing frames/0137.png (20/187)
  processing frames/0044.png (21/187)
  processing frames/0072.png (22/187)
  processing frames/0119.png (23/187)
  processing frames/0000.png (24/187)
  processing frames/0160.png (25/187)
  processing frames/0078.png (26/187)
  pr

Plane equation: -0.22x + 0.86y + 0.47z + -0.05 = 0
1.7201
50798.855
5.8039255
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.19x + 0.89y + 0.41z + -0.05 = 0
2.1768446
67291.7
7.3764544
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.18x + 0.89y + 0.42z + -0.05 = 0
1.9657362
58347.965
7.21905
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.17x + 0.88y + 0.44z + -0.05 = 0
1.8558233
52691.465
6.5434914
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.15x + 0.87y + 0.48z + -0.05 = 0
1.8143799
52018.277
5.7669926
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.21x + 0.88y + 0.43z + -0.05 = 0
1.9942613
58516.613
6.580445
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.20x + 0.87y

Plane equation: -0.13x + 0.95y + 0.30z + -0.03 = 0
3.0999124
61409.266
7.4165783
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.13x + 0.93y + 0.35z + -0.04 = 0
2.8633857
49765.645
7.78805
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.18x + 0.96y + 0.19z + -0.03 = 0
3.8374267
86342.1
12.299444
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.06x + 0.91y + 0.42z + -0.05 = 0
1.9024694
40998.21
6.5283785
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.00x + 0.93y + 0.36z + -0.05 = 0
1.9744142
50179.74
6.5210834
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.10x + 0.97y + 0.22z + -0.04 = 0
2.858146
75083.49
9.420766
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: -0.07x + 0.96y +

Plane equation: 0.00x + 0.90y + 0.44z + -0.05 = 0
2.0469348
68178.29
5.8939524
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.02x + 0.90y + 0.44z + -0.05 = 0
1.8026167
66709.21
6.0169535
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.02x + 0.90y + 0.44z + -0.05 = 0
2.0535588
70061.01
5.767822
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.04x + 0.90y + 0.44z + -0.05 = 0
2.1367793
70867.62
5.671394
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.03x + 0.90y + 0.44z + -0.05 = 0
2.108433
73276.12
6.038995
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.02x + 0.90y + 0.44z + -0.04 = 0
2.3631146
73164.234
5.649339
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.02x + 0.89y + 0.45z 

Plane equation: 0.03x + 0.90y + 0.44z + -0.04 = 0
2.2919621
72762.43
5.585601
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.04x + 0.89y + 0.45z + -0.04 = 0
2.389395
72712.65
5.4136286
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.03x + 0.89y + 0.45z + -0.04 = 0
2.297802
71064.914
5.501511
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.03x + 0.89y + 0.45z + -0.04 = 0
2.3231394
70354.81
5.5597234
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.03x + 0.90y + 0.44z + -0.04 = 0
2.3003557
71131.31
5.680564
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.03x + 0.89y + 0.45z + -0.04 = 0
2.3923903
69384.195
5.50579
[[440.692   0.    354.835]
 [  0.    440.692 232.086]
 [  0.      0.      1.   ]]
Plane equation: 0.03x + 0.89y + 0.45z +

**Download a pretrained model from Detectron2 Model Zoo for Faster-RCNN**

In [8]:
cfg = get_cfg()

# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2  # set threshold for this model

# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)

### Define all the key functions

In [9]:
# define a function which return the bottom center of every bbox
def mid_point(img,img_depth,person,idx):
  #get the coordinates
  x1,y1,x2,y2 = person[idx]
  _ = cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,255), 2)
  
  #compute bottom center of bbox
  x_mid = int((x1+x2)/2)
  y_mid = int(y2)
  mid   = (x_mid-1,y_mid-1)
  print("mid=",mid)
  print("img_depth",img_depth.shape)
  z_mid = img_depth[(y_mid-1,x_mid-1)]
  mid3d = (x_mid,y_mid,z_mid)
    
  _ = cv2.circle(img, mid, 5, (0, 0, 255), -1)
  cv2.putText(img, str(z_mid) + " m", mid, cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2, cv2.LINE_AA)
  return mid3d

# define a function which computes euclidean distance between two midpoints
from scipy.spatial import distance
def compute_distance(midpoints,num):
  dist = np.zeros((num,num))
  for i in range(num):
    for j in range(i+1,num):
      if i!=j:
        dst = distance.euclidean(midpoints[i], midpoints[j])
        dist[i][j]=dst
  return dist


# Finds pairs of people who are close together
def find_closest(dist,num,thresh):
  p1=[]
  p2=[]
  d=[]
  for i in range(num):
    for j in range(i,num):
      if( (i!=j) & (dist[i][j]<=thresh)):
        p1.append(i)
        p2.append(j)
        d.append(dist[i][j])
  return p1,p2,d


# Given pairs of people who are close, color them red
def change_2_red(img,img_depth,person,p1,p2):
  mid1 = []
  mid2 = []
  for p in p1:
    mid1.append(mid_point(img,img_depth,person,p))
  for pp in p2:
    mid2.append(mid_point(img,img_depth,person,pp))
  for inx in range(len(mid1)):
      #print("mid1",mid1[inx][:2])
      _ = cv2.line(img, mid1[inx][:2], mid2[inx][:2], (0,255,0), thickness=2, lineType=8, shift=0)
  
  risky = np.unique(p1+p2)
  for i in risky:
    x1,y1,x2,y2 = person[i]
    _ = cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 2)  
  return img


# Main function to find closest people
def find_closest_people(name,name_depth,thresh,savedir):

  img = cv2.imread(name)
  depth = np.load(name_depth)
  print("NPY file shape")
  print(depth.shape)
  original_height, original_width,_ = img.shape # (1920,1080) #input_image.size
  print("Original width, original_height:")
  print(original_width)
  print(original_height)
  disp_resized = torch.nn.functional.interpolate(
                    torch.from_numpy(depth).unsqueeze(0).unsqueeze(0), (original_height, original_width), mode="bilinear", align_corners=False)
  img_depth = disp_resized.squeeze().cpu().numpy()

  print("Depth resized shape, input to algorithms")
  print(disp_resized.shape)
  outputs = predictor(img)
  classes=outputs['instances'].pred_classes.cpu().numpy()
  bbox=outputs['instances'].pred_boxes.tensor.cpu().numpy()
  ind = np.where(classes>-1)[0]
  person=bbox[ind]
  midpoints = [mid_point(img,img_depth,person,i) for i in range(len(person))]
  #num = len(midpoints)
  #dist= compute_distance(midpoints,num)
  #p1,p2,d=find_closest(dist,num,thresh)
  #img = change_2_red(img,img_depth,person,p1,p2)
  cv2.imwrite(savedir+'/'+name,img)
  #print(savedir+'/'+name)
  return 0

**Fetch all the frames of the video sequence**

In [10]:
frames=[]
for file in os.listdir(frames_folder):
    if file.endswith(".png"):
        frames.append(frames_folder+"/"+file)
frames.sort()

**Fetch all the frame depths of the video sequence**

In [11]:
frame_depths=[]
for file in os.listdir(depths_folder):
    if file.endswith(".npy"):
        frame_depths.append(depths_folder+"/"+file)
frame_depths.sort()


### Main loop to get results

In [12]:
#from tqdm import tqdm
thresh=110
with tqdm(total=len(frames)) as pbar:
    for i in range(len(frames)):
        find_closest_people(frames[i],frame_depths[i],thresh,result_folder)
        pbar.update(1)
    
#_ = [find_closest_people(frames[i],thresh,'frames2') for i in tqdm(range(len(frames))) ]

  0%|          | 0/187 [00:00<?, ?it/s]

NPY file shape
(480, 720)
Original width, original_height:
720
480
Depth resized shape, input to algorithms
torch.Size([1, 1, 480, 720])


/home/awsgui/detectron2/detectron2/modeling/roi_heads/fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()
  1%|          | 2/187 [00:00<00:33,  5.57it/s]

mid= (380, 472)
img_depth (480, 720)
mid= (376, 464)
img_depth (480, 720)
NPY file shape
(480, 720)
Original width, original_height:
720
480
Depth resized shape, input to algorithms
torch.Size([1, 1, 480, 720])
mid= (380, 472)
img_depth (480, 720)
mid= (376, 464)
img_depth (480, 720)
NPY file shape
(480, 720)
Original width, original_height:
720
480
Depth resized shape, input to algorithms
torch.Size([1, 1, 480, 720])


  2%|▏         | 4/187 [00:00<00:26,  6.95it/s]

...

mid= (616, 188)
img_depth (480, 720)
mid= (367, 472)
img_depth (480, 720)
mid= (379, 451)
img_depth (480, 720)
NPY file shape
(480, 720)
Original width, original_height:
720
480
Depth resized shape, input to algorithms
torch.Size([1, 1, 480, 720])
mid= (614, 187)
img_depth (480, 720)
mid= (371, 472)
img_depth (480, 720)
NPY file shape
(480, 720)
Original width, original_height:
720
480
Depth resized shape, input to algorithms
torch.Size([1, 1, 480, 720])


 99%|█████████▉| 186/187 [00:23<00:00,  8.20it/s]

mid= (615, 188)
img_depth (480, 720)
mid= (369, 473)
img_depth (480, 720)
NPY file shape
(480, 720)
Original width, original_height:
720
480
Depth resized shape, input to algorithms
torch.Size([1, 1, 480, 720])
mid= (615, 187)
img_depth (480, 720)
mid= (371, 473)
img_depth (480, 720)
NPY file shape
(480, 720)
Original width, original_height:
720
480
Depth resized shape, input to algorithms
torch.Size([1, 1, 480, 720])


100%|██████████| 187/187 [00:23<00:00,  8.06it/s]

mid= (614, 187)
img_depth (480, 720)
mid= (370, 472)
img_depth (480, 720)


## Save results

**Main file with highlighed results**

In [13]:
frames=[]
for file in os.listdir(result_folder+"/frames/"):
    if file.endswith(".png"):
        frames.append(file)
frames.sort()

frame_array=[]
with tqdm(total=len(frames)) as pbar:
    for i in range(len(frames)):

        #reading each files
        img = cv2.imread(result_folder+'/frames/'+frames[i])
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

        height, width, layers = img.shape
        size = (width,height)

        #inserting the frames into an image array
        frame_array.append(img)
        pbar.update(1)

out = cv2.VideoWriter(result_folder+'/result.mp4',cv2.VideoWriter_fourcc(*'DIVX'), FPS, size)
 
with tqdm(total=len(frames)) as pbar:
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
        pbar.update(1)
out.release()


100%|██████████| 187/187 [00:00<00:00, 212.39it/s]


**Depth map video**

In [14]:
frames_depth=[]
for file in os.listdir(depths_folder):
    if file.endswith(".png"):
        frames_depth.append(file)
frames_depth.sort()

frame_array=[]
with tqdm(total=len(frames_depth)) as pbar:
    for i in range(len(frames_depth)):

        #reading each files
        img = cv2.imread(depths_folder+'/'+frames_depth[i])
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        

        height, width, layers = img.shape
        size = (width,height)

        #inserting the frames into an image array
        frame_array.append(img)
        pbar.update(1)

out = cv2.VideoWriter(result_folder+'/depth_result.mp4',cv2.VideoWriter_fourcc(*'DIVX'), FPS, size)
 
with tqdm(total=len(frames_depth)) as pbar:
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
        pbar.update(1)
out.release()

100%|██████████| 187/187 [00:00<00:00, 350.33it/s]
